In [1]:
#Importing libraries
import numpy as np
import pandas as pd
import tensorflow as tf
import seaborn as sn
import math
import cv2
import os
import PIL
import pathlib

from tensorflow import keras
from sklearn.metrics import confusion_matrix , classification_report
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
#Importing and reading training dataset
df_train = pd.read_csv('train.csv')
df_train.sample(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
350,351,0,3,"Odahl, Mr. Nils Martin",male,23.0,0,0,7267,9.2250,NaN,S
20,21,0,2,"Fynney, Mr. Joseph J",male,35.0,0,0,239865,26.0000,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C
74,75,1,3,"Bing, Mr. Lee",male,32.0,0,0,1601,56.4958,NaN,S
134,135,0,2,"Sobey, Mr. Samuel James Hayden",male,25.0,0,0,C.A. 29178,13.0000,NaN,S
279,280,1,3,"Abbott, Mrs. Stanton (Rosa Hunt)",female,35.0,1,1,C.A. 2673,20.2500,NaN,S
838,839,1,3,"Chip, Mr. Chang",male,32.0,0,0,1601,56.4958,NaN,S
698,699,0,1,"Thayer, Mr. John Borland",male,49.0,1,1,17421,110.8833,C68,C
502,503,0,3,"O'Sullivan, Miss. Bridget Mary",female,NaN,0,0,330909,7.6292,NaN,Q
627,628,1,1,"Longley, Miss. Gretchen Fiske",female,21.0,0,0,13502,77.9583,D9,S


In [3]:
#Importing and reading test dataset
df_test = pd.read_csv('test.csv')
df_test.sample(10)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
354,1246,3,"Dean, Miss. Elizabeth Gladys Millvina""""",female,0.17,1,2,C.A. 2315,20.5750,NaN,S
218,1110,1,"Widener, Mrs. George Dunton (Eleanor Elkins)",female,50.00,1,1,113503,211.5000,C80,C
38,930,3,"Sap, Mr. Julius",male,25.00,0,0,345768,9.5000,NaN,S
362,1254,2,"Ware, Mrs. John James (Florence Louise Long)",female,31.00,0,0,CA 31352,21.0000,NaN,S
348,1240,2,"Giles, Mr. Ralph",male,24.00,0,0,248726,13.5000,NaN,S
11,903,1,"Jones, Mr. Charles Cresson",male,46.00,0,0,694,26.0000,NaN,S
30,922,2,"Louch, Mr. Charles Alexander",male,50.00,1,0,SC/AH 3085,26.0000,NaN,S
378,1270,1,"Hipkins, Mr. William Edward",male,55.00,0,0,680,50.0000,C39,S
10,902,3,"Ilieff, Mr. Ylio",male,NaN,0,0,349220,7.8958,NaN,S
408,1300,3,"Riordan, Miss. Johanna Hannah""""",female,NaN,0,0,334915,7.7208,NaN,Q


In [4]:
#Importing and reading sample solution framework
df_outex = pd.read_csv('gender_submission.csv')
df_outex.sample(10)

,PassengerId,Survived
111,1003,1
105,997,0
319,1211,0
247,1139,0
233,1125,0
187,1079,0
345,1237,1
336,1228,0
87,979,1
322,1214,0


In [5]:
#Checking datatypes of training set
df_train.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [6]:
df_train.sample(21)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
450,451,0,2,"West, Mr. Edwy Arthur",male,36.00,1,2,C.A. 34651,27.7500,NaN,S
446,447,1,2,"Mellinger, Miss. Madeleine Violet",female,13.00,0,1,250644,19.5000,NaN,S
45,46,0,3,"Rogers, Mr. William John",male,NaN,0,0,S.C./A.4. 23567,8.0500,NaN,S
469,470,1,3,"Baclini, Miss. Helene Barbara",female,0.75,2,1,2666,19.2583,NaN,C
510,511,1,3,"Daly, Mr. Eugene Patrick",male,29.00,0,0,382651,7.7500,NaN,Q
727,728,1,3,"Mannion, Miss. Margareth",female,NaN,0,0,36866,7.7375,NaN,Q
815,816,0,1,"Fry, Mr. Richard",male,NaN,0,0,112058,0.0000,B102,S
637,638,0,2,"Collyer, Mr. Harvey",male,31.00,1,1,C.A. 31921,26.2500,NaN,S
692,693,1,3,"Lam, Mr. Ali",male,NaN,0,0,1601,56.4958,NaN,S
696,697,0,3,"Kelly, Mr. James",male,44.00,0,0,363592,8.0500,NaN,S


In [7]:
#Mapping objects to int and filling the NaN values in training set
df_train['Sex'] = df_train['Sex'].map({'male': 1, 'female': 0})
df_train['Age'] = df_train['Age'].fillna(0)
df_train['Embarked'] = df_train['Embarked'].map({'S':1, 'C': 0})
df_train['Embarked'] = df_train['Embarked'].fillna(0.5)
df_train.sample(21)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
183,184,1,2,"Becker, Master. Richard F",1,1.0,2,1,230136,39.0000,F4,1.0
768,769,0,3,"Moran, Mr. Daniel J",1,0.0,1,0,371110,24.1500,NaN,0.5
847,848,0,3,"Markoff, Mr. Marin",1,35.0,0,0,349213,7.8958,NaN,0.0
324,325,0,3,"Sage, Mr. George John Jr",1,0.0,8,2,CA. 2343,69.5500,NaN,1.0
74,75,1,3,"Bing, Mr. Lee",1,32.0,0,0,1601,56.4958,NaN,1.0
388,389,0,3,"Sadlier, Mr. Matthew",1,0.0,0,0,367655,7.7292,NaN,0.5
325,326,1,1,"Young, Miss. Marie Grice",0,36.0,0,0,PC 17760,135.6333,C32,0.0
75,76,0,3,"Moen, Mr. Sigurd Hansen",1,25.0,0,0,348123,7.6500,F G73,1.0
706,707,1,2,"Kelly, Mrs. Florence ""Fannie""",0,45.0,0,0,223596,13.5000,NaN,1.0
531,532,0,3,"Toufik, Mr. Nakli",1,0.0,0,0,2641,7.2292,NaN,0.0


In [8]:
#Mapping objects to int and filling the NaN values in test set
df_test['Sex'] = df_test['Sex'].map({'male': 1, 'female': 0})
df_test['Age'] = df_test['Age'].fillna(0)
df_test['Embarked'] = df_test['Embarked'].map({'S':1, 'C': 0})
df_test['Embarked'] = df_test['Embarked'].fillna(0.5)
df_test.sample(21)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
318,1210,3,"Jonsson, Mr. Nils Hilding",1,27.0,0,0,350408,7.8542,NaN,1.0
17,909,3,"Assaf, Mr. Gerios",1,21.0,0,0,2692,7.2250,NaN,0.0
359,1251,3,"Lindell, Mrs. Edvard Bengtsson (Elin Gerda Per...",0,30.0,1,0,349910,15.5500,NaN,1.0
310,1202,3,"Cacic, Mr. Jego Grga",1,18.0,0,0,315091,8.6625,NaN,1.0
119,1011,2,"Chapman, Mrs. John Henry (Sara Elizabeth Lawry)",0,29.0,1,0,SC/AH 29037,26.0000,NaN,1.0
174,1066,3,"Asplund, Mr. Carl Oscar Vilhelm Gustafsson",1,40.0,1,5,347077,31.3875,NaN,1.0
327,1219,1,"Rosenshine, Mr. George (Mr George Thorne"")""",1,46.0,0,0,PC 17585,79.2000,NaN,0.0
82,974,1,"Case, Mr. Howard Brown",1,49.0,0,0,19924,26.0000,NaN,1.0
72,964,3,"Nieminen, Miss. Manta Josefina",0,29.0,0,0,3101297,7.9250,NaN,1.0
83,975,3,"Demetri, Mr. Marinko",1,0.0,0,0,349238,7.8958,NaN,1.0


In [9]:
#Testing for correlation
Pclass = df_train['Pclass']
Survived = df_train['Survived']
r = np.corrcoef(Pclass, Survived)[0, 1]
r

-0.3384810359610147

In [10]:
#Testing for correlation
Sex = df_train['Sex']
Survived = df_train['Survived']
r = np.corrcoef(Sex, Survived)[0, 1]
r

-0.5433513806577551

In [11]:
#Testing for correlation
Age = df_train['Age']
Survived = df_train['Survived']
r = np.corrcoef(Age, Survived)[0, 1]
r

0.010539215871285656

In [12]:
#Testing for correlation
SibSp = df_train['SibSp']
Survived = df_train['Survived']
r = np.corrcoef(SibSp, Survived)[0, 1]
r

-0.03532249888573558

In [13]:
#Testing for correlation
Parch = df_train['Parch']
Survived = df_train['Survived']
r = np.corrcoef(Parch, Survived)[0, 1]
r

0.08162940708348365

In [14]:
#Testing for correlation
Embarked = df_train['Embarked']
Survived = df_train['Survived']
r = np.corrcoef(Embarked, Survived)[0, 1]
r

-0.17124874580816557

In [15]:
#Scaling the required columns in training set
cols_to_scale = ['Pclass', 'SibSp', 'Parch']

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df_train[cols_to_scale] = scaler.fit_transform(df_train[cols_to_scale])

In [16]:
#Scaling the required columns in test set
cols_to_scale = ['Pclass', 'SibSp', 'Parch']

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df_test[cols_to_scale] = scaler.fit_transform(df_test[cols_to_scale])

In [17]:
#Defining training and test sets
y_train = df_train["Survived"]

features = ["Pclass", "Embarked", "Sex", "Parch"]
x_train = df_train[features]
x_test = df_test[features]

In [19]:
x_train.sample(21)

,Pclass,Embarked,Sex,Parch
748,0.0,1.0,1,0.000000
568,1.0,0.0,1,0.000000
690,0.0,1.0,1,0.000000
397,0.5,1.0,1,0.000000
653,1.0,0.5,0,0.000000
551,0.5,1.0,1,0.000000
556,0.0,0.0,0,0.000000
586,0.5,1.0,1,0.000000
191,0.5,1.0,1,0.000000
223,1.0,1.0,1,0.000000


In [21]:
#Defining ANN Model
def ANN(x_train, y_train, x_test, loss, weights):
    model = keras.Sequential([
        keras.layers.Dense(100,input_shape=(4,), activation='relu'),
        keras.layers.Dense(15, activation='relu'),
        keras.layers.Dense(1, activation='sigmoid') #SIGMOID FOR BINARY CLASSIFICATION AND SOFTMAX FOR MULTICLASS CLASSIFICATION
    ])

    model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])
    
    if weights == -1:
        model.fit(x_train, y_train, epochs=100)
    else:
        model.fit(x_train, y_train, epochs=100, class_weight = weights)
    
    y_preds = model.predict(x_test)
    y_preds = np.round(y_preds)
    
    return y_preds

In [22]:
y_preds = ANN(x_train, y_train, x_test, 'binary_crossentropy', -1)

Epoch 1/100
28/28 [==============================] - 2s 4ms/step - loss: 0.6088 - accuracy: 0.6195
Epoch 2/100
28/28 [==============================] - 0s 3ms/step - loss: 0.5614 - accuracy: 0.7160
Epoch 3/100
28/28 [==============================] - 0s 3ms/step - loss: 0.5271 - accuracy: 0.7688
Epoch 4/100
28/28 [==============================] - 0s 3ms/step - loss: 0.5036 - accuracy: 0.7856
Epoch 5/100
28/28 [==============================] - 0s 3ms/step - loss: 0.4916 - accuracy: 0.7868
Epoch 6/100
28/28 [==============================] - 0s 3ms/step - loss: 0.4816 - accuracy: 0.7868
Epoch 7/100
28/28 [==============================] - 0s 3ms/step - loss: 0.4745 - accuracy: 0.7868
Epoch 8/100
28/28 [==============================] - 0s 3ms/step - loss: 0.4679 - accuracy: 0.7856
Epoch 9/100
28/28 [==============================] - 0s 3ms/step - loss: 0.4645 - accuracy: 0.7924
Epoch 10/100
28/28 [==============================] - 0s 3ms/step - loss: 0.4608 - accuracy: 0.7879
Epoch 11/

28/28 [==============================] - 0s 3ms/step - loss: 0.4331 - accuracy: 0.8092
Epoch 84/100
28/28 [==============================] - 0s 3ms/step - loss: 0.4339 - accuracy: 0.8148
Epoch 85/100
28/28 [==============================] - 0s 3ms/step - loss: 0.4391 - accuracy: 0.8103
Epoch 86/100
28/28 [==============================] - 0s 3ms/step - loss: 0.4340 - accuracy: 0.8103
Epoch 87/100
28/28 [==============================] - 0s 3ms/step - loss: 0.4333 - accuracy: 0.8103
Epoch 88/100
28/28 [==============================] - 0s 3ms/step - loss: 0.4333 - accuracy: 0.8114
Epoch 89/100
28/28 [==============================] - 0s 3ms/step - loss: 0.4346 - accuracy: 0.8070
Epoch 90/100
28/28 [==============================] - 0s 3ms/step - loss: 0.4355 - accuracy: 0.8137
Epoch 91/100
28/28 [==============================] - 0s 5ms/step - loss: 0.4346 - accuracy: 0.8148
Epoch 92/100
28/28 [==============================] - 0s 3ms/step - loss: 0.4340 - accuracy: 0.8126
Epoch 93/100


In [34]:
#Converting float output to int
def float_to_int(float_number):
  
  return int(float_number)

y_preds_int = list(map(float_to_int, y_preds))

y_preds_int

[0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,


In [35]:
#Converting the output to desired csv
output = pd.DataFrame({'PassengerId': df_test.PassengerId, 'Survived': y_preds_int})
output.to_csv('titanic_ml_from_disaster_final.csv', index=False)

In [36]:
#Importing and reading the output dataset
df = pd.read_csv('titanic_ml_from_disaster_final.csv')
df.sample(25)

,PassengerId,Survived
10,902,0
33,925,0
377,1269,0
297,1189,0
100,992,1
228,1120,0
61,953,0
329,1221,0
220,1112,1
157,1049,0
